# Hartree-Fock implementation from scratch

In [1]:
import numpy as np
from itertools import combinations

In [2]:
# convert a triangular matrix into a symmetric one
def symmetrize(matrix):
    return matrix + matrix.T - np.diag(matrix.diagonal())

# Yoshimine Sort 
Yoshimine sort is a way to store two-electron integrals more efficiently. Each integral $(ij|kl)$ depends on four indices, but many of them are the same because of symmetry — for example, Instead of saving all these duplicates, Yoshimine sort turns the four indices into one unique integer key. 
That key always gives the same number for any symmetric permutation of $(ij∣kl)$, so you only store each integral once and can find it quickly when building the Fock matrix.


In [3]:
# Return compound index given four indices using Yoshimine sort
def yoshimine(a, b, c, d): 
    if a > b: ab = a*(a+1)/2 + b
    else: ab = b*(b+1)/2 + a
    if c > d: cd = c*(c+1)/2 + d
    else: cd = d*(d+1)/2 + c
    if ab > cd: abcd = ab*(ab+1)/2 + cd
    else: abcd = cd*(cd+1)/2 + ab

    return abcd


In [4]:
# Load Two Electron Integrals (TEI) from file
TEI = np.genfromtxt('./two_electron_int.txt') 
two_e = {yoshimine(row[0], row[1], row[2], row[3]) : row[4] for row in TEI}

In [5]:
# Return value of two electron integral
def two_electron_integral(a, b, c, d): 
    return two_e.get(yoshimine(a, b, c, d))

# Density Matrix Construction
In restricted Hartree–Fock (RHF), the density matrix represents how electrons occupy the atomic-orbital basis functions.
It is computed from the molecular-orbital (MO) coefficients as
$$
D_{\mu \nu} = 2 \sum_{m=1}^{N_\text{occ}} C_{\mu m} C_{\nu m}
$$


In [6]:
def make_density_matrix(C, D, dim, N):
    Dold = np.zeros((dim, dim))
    for mu in range(0, dim):
        for nu in range(0, dim):
            Dold[mu,nu] = D[mu, nu]
            D[mu,nu] = 0
            for m in range(0, int(N/2)):
                D[mu,nu] = D[mu,nu] + 2*C[mu,m]*C[nu,m]

    return D, Dold

# Fock matrix
The Fock matrix is the central matrix in the Hartree–Fock method — it represents the effective one-electron Hamiltonian that each electron “feels” in the average field of all the others. In other words, instead of solving the full, many-electron Schrödinger equation (which is impossible to do exactly for large systems), Hartree–Fock approximates it by making each electron move in an average potential created by all other electrons. The Fock matrix collects this information in the atomic orbital (AO) basis. Mathematically, it defines the Fock operator ,whose matrix form in an AO basis is:

$$
F_{\mu \nu} = H^{core}_{\mu \nu} + \sum_{\lambda,\sigma} P_{\lambda \sigma} [ (\mu \nu|\lambda \sigma) - \dfrac{1}{2} (\mu \nu|\nu \sigma) ] 
$$

In [7]:
def make_fock_matrix(H, P, dim):
    F = np.zeros((dim, dim))
    for i in range(0, dim):
        for j in range(0, dim):
            F[i,j] = H[i,j]
            for k in range(0, dim):
                for l in range(0, dim):
                    F[i,j] = F[i,j] + P[k,l]*(two_electron_integral(i + 1, j + 1, k + 1, l + 1) - 0.5 * two_electron_integral(i + 1, k + 1, j + 1, l + 1))
    
    return F

In [8]:
# Calculate change in density matrix using Root Mean Square Deviation (RMSD)
def delta_p(D, Dold, dim):
    DELTA = 0.0
    for i in range(0, dim):
        for j in range(0, dim):
            DELTA = DELTA + ((D[i,j] - Dold[i,j])**2)

    return DELTA**0.5

In [9]:
# Calculate energy at iteration
def current_energy(D, H, F, dim):
    EN = 0
    for mu in range(0, dim):
        for nu in range(0, dim):
            EN += 0.5*D[mu,nu]*(H[mu,nu] + F[mu,nu])
            
    return EN

# $HHe^+$ System


In [10]:
N = 2 # The number of electrons in our system 
E_NUC = np.genfromtxt('./e_nuc.txt',dtype=float, delimiter=',') # nuclear repulsion, 
S_raw = np.genfromtxt('./s.txt',dtype=None) # overlap matrix, 
T_raw = np.genfromtxt('./t.txt',dtype=None) # kinetic energy matrix,
V_raw = np.genfromtxt('./v.txt',dtype=None) # potential energy matrix

dim = 2 # dim is the number of basis functions 
S = np.zeros((dim, dim)) 
T = np.zeros((dim, dim))
V = np.zeros((dim, dim))

for i in S_raw: S[i[0]-1, i[1]-1] = i[2] 
for i in T_raw: T[i[0]-1, i[1]-1] = i[2] 
for i in V_raw: V[i[0]-1, i[1]-1] = i[2]

In [11]:
S = symmetrize(S) 
V = symmetrize(V) 
T = symmetrize(T)

Hcore = T + V 

# Symmetric orthogonalization
eigvals, eigvecs = np.linalg.eigh(S)
S_inv_sqrt = eigvecs @ np.diag(eigvals**-0.5) @ eigvecs.T  # X = S^{-1/2}

# Initialize density and SCF loop control variables
P_new = np.zeros((dim, dim)) 
delta = 1.0 
iteration = 0 


In [12]:
while delta > 1e-5:
    iteration += 1
    
    F = make_fock_matrix(Hcore, P_new, dim)      
    Fprime = np.dot(np.transpose(S_inv_sqrt), np.dot(F, S_inv_sqrt)) 
    
    E, C_prime = np.linalg.eigh(Fprime)        
    C = np.dot(S_inv_sqrt, C_prime)
    
    P_new, P_old = make_density_matrix(C, P_new, dim, N)
    
    delta = delta_p(P_new, P_old, dim)    
    
    print(f" iteration {iteration}: E = {current_energy(P_new, Hcore, F, dim) + E_NUC:.6f}")

print("TOTAL E(SCF) = {} hartrees".format(current_energy(P_new, Hcore, F, dim) + E_NUC))


 iteration 1: E = -3.345374
 iteration 2: E = -2.418618
 iteration 3: E = -2.439621
 iteration 4: E = -2.443370
 iteration 5: E = -2.444016
 iteration 6: E = -2.444127
 iteration 7: E = -2.444146
 iteration 8: E = -2.444149
TOTAL E(SCF) = -2.444149490118208 hartrees


# Full Configuration Interaction

In [13]:
eri_AO = np.zeros((dim, dim, dim, dim))
for mu in range(dim):
    for nu in range(dim):
        for la in range(dim):
            for si in range(dim):
                eri_AO[mu,nu,la,si] = two_e.get(yoshimine(mu,nu,la,si), 0.0)

In [14]:
# AO -> MO integral transformation
h_MO = C.T @ Hcore @ C

K = C.shape[0]
eri_MO = np.zeros((K,K,K,K))
for p in range(K):
    for q in range(K):
        Tpq = np.tensordot(eri_AO, C[:,p], axes=(0,0))        
        Tpq = np.tensordot(Tpq,     C[:,q], axes=(0,0))        
        for r in range(K):
            Tpqr = np.tensordot(Tpq, C[:,r], axes=(0,0))       
            for s in range(K):
                eri_MO[p,q,r,s] = np.tensordot(Tpqr, C[:,s], axes=(0,0))

In [15]:
# Build spin-orbital integrals and antisymmetrize

M = 2*K
h_so = np.zeros((M, M))
h_so[:K, :K] = h_MO      # alpha block
h_so[K:, K:] = h_MO      # beta block


g_asym = np.zeros((M, M, M, M))
for p in range(M):
    P, sp_p = p % K, p // K  # spatial index and spin (0=α, 1=β)
    for q in range(M):
        Q, sp_q = q % K, q // K
        for r in range(M):
            R, sp_r = r % K, r // K
            for s in range(M):
                S_, sp_s = s % K, s // K
                # Spin selection rules:
                if sp_p == sp_r and sp_q == sp_s:   # spin-conserving pair interaction
                    val = eri_MO[P, Q, R, S_]
                    # Subtract exchange only when both pairs have same spins
                    if sp_p == sp_s and sp_q == sp_r:
                        val -= eri_MO[P, Q, S_, R]
                    g_asym[p, q, r, s] = val


In [16]:
# Enumerate the determinant basis

Nalpha = Nbeta = N // 2  # closed-shell assumption

def bits_from_occ(occ):
    b = 0
    for i in occ: b |= (1 << i)
    return b

alpha_occ = list(combinations(range(K), Nalpha))
beta_occ  = list(combinations(range(K), Nbeta))

dets = [(bits_from_occ(oa), bits_from_occ(ob)) for oa in alpha_occ for ob in beta_occ]
ndet = len(dets)
print(f"Number of determinants = {ndet}")


Number of determinants = 4


In [17]:
# 5. Slater–Condon helpers

def occ_list(bits):
    occ, i = [], 0
    while bits:
        if bits & 1:
            occ.append(i)
        bits >>= 1
        i += 1
    return occ

def excitation_info(a_bits, b_bits):
    """Return excitation degree and which orbitals differ."""
    removed = [i for i in occ_list(a_bits) if not ((b_bits >> i) & 1)]
    added   = [i for i in occ_list(b_bits) if not ((a_bits >> i) & 1)]
    return len(removed), removed, added

def phase_single(bits, b, a):
    """Phase factor for single excitation (b->a)."""
    if a == b: return 1
    low, high = (a,b) if a < b else (b,a)
    mask = ((1 << high) - 1) ^ ((1 << low) - 1)
    n_cross = bin(bits & mask).count("1")
    return -1 if n_cross % 2 else 1

# Slater–Condon matrix elements
Diagonal::
$$
H_{II} = 
\sum_{i \in \mathrm{occ}} h_{ii}
+ \tfrac{1}{2} \sum_{i,j \in \mathrm{occ}} 
\langle ij \| ij \rangle
$$

Single:
$$
H_{IJ} = \pm \left(
h_{ab} + \sum_{k \in \mathrm{occ}}
\langle ak \| bk \rangle
\right)
$$

Double:
$$
H_{IJ} = \pm \langle ac \| bd \rangle
$$



In [18]:
# 6. Slater–Condon matrix elements

def diagonal_energy(alpha_bits, beta_bits):
    occ_alpha = occ_list(alpha_bits)
    occ_beta  = [i+K for i in occ_list(beta_bits)]
    occ = occ_alpha + occ_beta
    e1 = sum(h_so[p,p] for p in occ)
    e2 = 0.0
    # the 1/2 coefficient is self-contained here:
    for i,p in enumerate(occ):
        for q in occ[:i]:
            e2 += g_asym[p,q,p,q] + g_asym[q,p,q,p]
    return e1 + e2

def hij(Di, Dj):
    ai, bi = Di # Di = (alpha bits, beta bits)
    aj, bj = Dj
    da, rem_a, add_a = excitation_info(ai, aj)
    db, rem_b, add_b = excitation_info(bi, bj)
    deg = da + db
    
    """
    The degree of excitation is how many individual electron moves are required to turn one determinant into the other.
    If three or more electrons differ, By the Slater–Condon rules, all those matrix elements are zero because the Hamiltonian only contains one- and two-electron operators — it cannot connect determinants that differ by three or more orbitals.
    """
    if deg > 2:
        return 0.0
    
    if deg == 0:
        return diagonal_energy(ai, bi)
    
    # SINGLE excitation
    if deg == 1:
        if da == 1: # alpha-spin electron moved
            b, a = rem_a[0], add_a[0]
            ph = phase_single(ai, b, a)
            a_so, b_so = a, b
            occ_alpha = [i for i in occ_list(ai) if i != b]
            occ_beta  = [i+K for i in occ_list(bi)]
        else:
            b, a = rem_b[0], add_b[0]
            ph = phase_single(bi, b, a)
            a_so, b_so = a+K, b+K
            occ_alpha = [i for i in occ_list(ai)]
            occ_beta  = [i+K for i in occ_list(bi) if i != b]
        occ = occ_alpha + occ_beta
        val = h_so[a_so,b_so] + sum(g_asym[a_so,k,b_so,k] for k in occ)
        return ph * val
    
    # DOUBLE excitation
    if deg == 2:
        if da == 2:
            b,d = rem_a; a,c = add_a
            a_so,c_so = a,c
            b_so,d_so = b,d
        elif db == 2:
            b,d = rem_b; a,c = add_b
            a_so,c_so = a+K,c+K
            b_so,d_so = b+K,d+K
        else:
            # one alpha, one beta excitation
            b = rem_a[0]; a = add_a[0]
            d = rem_b[0]; c = add_b[0]
            a_so,c_so = a, c+K
            b_so,d_so = b, d+K
        return g_asym[a_so,c_so,b_so,d_so]


In [19]:
# 7. Build and diagonalize the CI Hamiltonian

H_CI = np.zeros((ndet, ndet))
for i, Di in enumerate(dets):
    for j, Dj in enumerate(dets[:i+1]):
        val = hij(Di, Dj)
        H_CI[i,j] = H_CI[j,i] = val

E_vals, C_vecs = np.linalg.eigh(H_CI)
E_elec_FCI = E_vals[0]
E_tot_FCI  = E_elec_FCI + E_NUC


print(f"FCI total energy     : {E_tot_FCI :.8f} hartrees")

FCI total energy     : -2.79121453 hartrees
